# Research Request - GTFS Digest: Add Rail and Ferry Operators. #1386
# Focus on Rail here. 

* Why is Amtrak "schedule_only"? I thought it had a real time component?
    * 3/3: no reports.calitp.org indicates it has no real time data. 
* Understand why Southern California Regional Rail Authority 	 & San Bernardino County Transportation Authority 	are still "vp_only" after all the other operators have been fixed.
* Visualize rail routes differently for operators with both? 

In [1]:
import _section1_utils as section1
import _section2_utils as section2
import geopandas as gpd
import merge_data
import merge_operator_data
import numpy as np
import pandas as pd
import geopandas as gpd
from segment_speed_utils import gtfs_schedule_wrangling, helpers
from segment_speed_utils.project_vars import COMPILED_CACHED_VIEWS, PROJECT_CRS
from shared_utils import catalog_utils, portfolio_utils, rt_dates
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
analysis_date_list = [rt_dates.DATES["feb2025"]]

In [4]:
analysis_date = rt_dates.DATES["feb2025"]

In [5]:
schd_vp_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.route_schedule_vp}.parquet"

In [6]:
EXPORT = GTFS_DATA_DICT.schedule_tables.route_typologies

In [7]:
route_typologies = pd.read_parquet(f"{SCHED_GCS}{EXPORT}_{analysis_date}.parquet")

In [8]:
schd_vp_df = pd.read_parquet(
    schd_vp_url,
    columns=[
        "schedule_gtfs_dataset_key",
        "caltrans_district",
        "organization_name",
        "name",
        "sched_rt_category",
        "service_date",
    ],
)

In [9]:
# Filter for Jan and Feb
schd_vp_df2 = schd_vp_df.loc[
    (schd_vp_df.service_date == "2025-02-12")
]

In [10]:
# Drop duplicates
schd_vp_df3 = (
    schd_vp_df2[
        [
            "schedule_gtfs_dataset_key",
            "organization_name",
            "service_date",
            "sched_rt_category",
            "caltrans_district",
        ]
    ]
    .drop_duplicates(subset=["organization_name"])
    .sort_values(by=["organization_name"])
)

In [11]:
schd_vp_df3.sched_rt_category.value_counts()

schedule_and_vp    104
schedule_only       86
vp_only              2
Name: sched_rt_category, dtype: int64

## Trying to undestand why Southern California Regional Rail Authority 	 & San Bernardino County Transportation Authority 	are still "vp_only" after all the other operators have been fixed.

In [12]:
vp_only_ops = list(
    schd_vp_df3.loc[
        schd_vp_df3.sched_rt_category == "vp_only"
    ].organization_name.unique()
)

In [13]:
vp_only_ops_df = schd_vp_df.loc[schd_vp_df.organization_name.isin(vp_only_ops)]

In [14]:
vp_only_ops_sched_keys = list(vp_only_ops_df.schedule_gtfs_dataset_key.unique())

In [15]:
vp_only_ops_df.groupby(["caltrans_district","organization_name", "sched_rt_category"]).agg(
    {"service_date": "max"}
)

service_date
caltrans_district          organization_name                              sched_rt_category             
07 - Los Angeles           San Bernardino County Transportation Authority schedule_only              NaT
                                                                          vp_only                    NaT
                                                                          schedule_and_vp            NaT
                           Southern California Regional Rail Authority    schedule_only       2024-06-12
                                                                          vp_only             2024-12-11
                                                                          schedule_and_vp     2024-06-12
07 - Los Angeles / Ventura San Bernardino County Transportation Authority schedule_only              NaT
                                                                          vp_only                    NaT
                                                                          schedule_and_vp            NaT
                           Southern California Regional Rail Authority    schedule_only              NaT
                                                                          vp_only             2025-02-12
                                                                          schedule_and_vp            NaT
08 - San Bernardino        San Bernardino County Transportation Authority schedule_only       2024-06-12
                                                                          vp_only             2025-02-12
                                                                          schedule_and_vp     2024-06-12
                           Southern California Regional Rail Authority    schedule_only              NaT
                                                                          vp_only                    NaT
                                                                          schedule_and_vp            NaT

## Go back to  `schedule_metrics_by_route_direction`

In [16]:
import sys

sys.path.append("../gtfs_funnel/")
import schedule_stats_by_route_direction

In [17]:
route_group_merge_cols = [
            "schedule_gtfs_dataset_key", 
            "route_id", 
            "direction_id"
        ]
        

### The two operators are already disappearing in the `trip_metrics` line 203. 

In [18]:
trip_metrics = schedule_stats_by_route_direction.assemble_scheduled_trip_metrics(
    analysis_date, GTFS_DATA_DICT
)

In [19]:
trip_metrics = trip_metrics.loc[
    trip_metrics.schedule_gtfs_dataset_key.isin(vp_only_ops_sched_keys)
]

In [20]:
vp_only_ops_sched_keys

['759ad28de7d4bb8b2bf9bb7d83655100', 'c4092405159366c705b62df938293a4e']

### Operators are already gone in `STOP_TIMES_FILE` in line 21.

In [21]:
STOP_TIMES_FILE = GTFS_DATA_DICT.rt_vs_schedule_tables.stop_times_direction

In [22]:
df = gpd.read_parquet(
        f"{RT_SCHED_GCS}{STOP_TIMES_FILE}_{analysis_date}.parquet"
    )

In [23]:
df = df.loc[
    df.schedule_gtfs_dataset_key.isin(vp_only_ops_sched_keys)
]

In [24]:
len(df)

0